In [3]:
print("Notebook is working")

Notebook is working


In [34]:
import requests

#Weather API endpoint
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": date,
    "end_date": date, 
    "hourly": "wind_direction_10m,wind_speed_10m",
    "timezone": "America/New_York"
}

response = requests.get(url, params = params)
data = response.json()

In [35]:
#Citizen's Bank Park (Phillies Stadium)
latitude = 39.9057
longitude = -75.1665

date = '2025-06-30'

In [36]:
print(data.keys())

dict_keys(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'elevation', 'hourly_units', 'hourly'])
